## Procedural Notes  

### Enzyme/Pathway Selection  
Kegg glycolysis/Glucogenesis-Reference Pathway:  

>Enzymes selected for each pathway are:  
>- Glycolysis: K00844, K11645, K01689, K16370  
>- Citric Acid Cycle: K01647, K00030, K01900, K00116  
>- Pentose Phosphate Pathway: K13937, K00036, K01807, K06859  



In [1]:
# Creating the genes, pathways and enzymes table with respective assigned columns
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
# to avoid conflict with previously built table
c.execute("""DROP TABLE genes""")
c.execute("""DROP TABLE pathways""")
c.execute("""DROP TABLE enzymes""")

# Caution: use INTEGER instead of INT for primary key generation
c.execute("""CREATE TABLE genes (id INTEGER PRIMARY KEY, name TEXT, description TEXT, organism TEXT, nucleotide_sequence TEXT
chromosome TEXT, start INT, end INT, strand VARCHAR(1), translated_sequence TEXT);""")

c.execute("""CREATE TABLE pathways (id INTEGER PRIMARY KEY, name TEXT, description TEXT);""")

c.execute("""CREATE TABLE enzymes (id INTEGER PRIMARY KEY, kegg_id TEXT, name TEXT, function TEXT, ec_number TEXT);""")

conn.commit()


In [2]:
# Inserting values into the pathway table
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
cmd = ''' INSERT INTO pathways (name, description) VALUES(?,?)'''

names = ["Glycolysis","Citric Acid Cycle","Pentose Phosphate"]
descriptions = ["the breakdown of glucose by enzymes, releasing energy and pyruvic acid." , "the sequence of reactions by which most living cells generate energy during the process of aerobic respiration. It takes place in the mitochondria, consuming oxygen, producing carbon dioxide and water as waste products, and converting ADP to energy-rich ATP","a metabolic pathway parallel to glycolysis, It generates NADPH and pentoses (5-carbon sugars) as well as ribose 5-phosphate, the last one a precursor for the synthesis of nucleotides"]


# Take in two list with equal length and insert the into the corresponding field
for i in range(len(names)):
    name = names[i]
    description = descriptions[i]
    temp_value = (name,description)
    c.execute(cmd,temp_value)
    conn.commit()


### Query the name and ec_number of the selected Enzymes from KEGG
1. Make a sample.list file in the current directory with the following lines  
K00844   
K11645   
K01689  
K16370  
K01647  
K00030  
K01900  
K00116  
K13937  
K00036  
K01807  
K06859  

2. In the terminal execute: `curl -g -s -S http://rest.kegg.jp/list/ko | grep -f sample.list | sed "s/cpd\://" > sample_extracted.table.txt`

3. Inspect the output file sample_extracted.table.txt  

In [3]:
# parse in the sample_extracted.table.txt for enzymes table insertion
def parse_extracted_table():
    filename = "sample_extracted.table.txt"
    in_file = open(filename, "rt")
    lines = []
    for line in in_file:
        lines.append(line.rstrip('\n'))
        print(line)
    return lines

lines = parse_extracted_table()



def get_values(lines):
    kegg_ids = []
    names = []
    functions = []
    ec_numbers = []
    for line in lines:
        line_ec = line.split("[")
        ec_number = line_ec[-1]
        line_2_part = line_ec[0].split(";")
        line_part1 = line_2_part[0].split()
        line_part2 = line_2_part[1].split()
#         print(line_part1)
#         print(line_part2)
        kegg_id = line_part1[0]
        temp_name = line_part1[1:]
        temp_name[:] = [''.join(temp_name[:])]
        name = temp_name[0]
        temp_function = line_part2[:]
        temp_function[:] = [" ".join(temp_function[:])]
        function = temp_function[0]     
        kegg_ids.append(kegg_id)
        names.append(name)
        functions.append(function)
        ec_numbers.append(ec_number)
    return kegg_ids,names,functions,ec_numbers

get_values(lines)





ko:K00030	IDH3; isocitrate dehydrogenase (NAD+) [EC:1.1.1.41]

ko:K00036	G6PD, zwf; glucose-6-phosphate 1-dehydrogenase [EC:1.1.1.49 1.1.1.363]

ko:K00116	mqo; malate dehydrogenase (quinone) [EC:1.1.5.4]

ko:K00844	HK; hexokinase [EC:2.7.1.1]

ko:K01647	CS, gltA; citrate synthase [EC:2.3.3.1]

ko:K01689	ENO, eno; enolase [EC:4.2.1.11]

ko:K01807	rpiA; ribose 5-phosphate isomerase A [EC:5.3.1.6]

ko:K01900	LSC2; succinyl-CoA synthetase beta subunit [EC:6.2.1.4 6.2.1.5]

ko:K06859	pgi1; glucose-6-phosphate isomerase, archaeal [EC:5.3.1.9]

ko:K11645	fbaB; fructose-bisphosphate aldolase, class I [EC:4.1.2.13]

ko:K13937	H6PD; hexose-6-phosphate dehydrogenase [EC:1.1.1.47 3.1.1.31]

ko:K16370	pfkB; 6-phosphofructokinase 2 [EC:2.7.1.11]



(['ko:K00030',
  'ko:K00036',
  'ko:K00116',
  'ko:K00844',
  'ko:K01647',
  'ko:K01689',
  'ko:K01807',
  'ko:K01900',
  'ko:K06859',
  'ko:K11645',
  'ko:K13937',
  'ko:K16370'],
 ['IDH3',
  'G6PD,zwf',
  'mqo',
  'HK',
  'CS,gltA',
  'ENO,eno',
  'rpiA',
  'LSC2',
  'pgi1',
  'fbaB',
  'H6PD',
  'pfkB'],
 ['isocitrate dehydrogenase (NAD+)',
  'glucose-6-phosphate 1-dehydrogenase',
  'malate dehydrogenase (quinone)',
  'hexokinase',
  'citrate synthase',
  'enolase',
  'ribose 5-phosphate isomerase A',
  'succinyl-CoA synthetase beta subunit',
  'glucose-6-phosphate isomerase, archaeal',
  'fructose-bisphosphate aldolase, class I',
  'hexose-6-phosphate dehydrogenase',
  '6-phosphofructokinase 2'],
 ['EC:1.1.1.41]',
  'EC:1.1.1.49 1.1.1.363]',
  'EC:1.1.5.4]',
  'EC:2.7.1.1]',
  'EC:2.3.3.1]',
  'EC:4.2.1.11]',
  'EC:5.3.1.6]',
  'EC:6.2.1.4 6.2.1.5]',
  'EC:5.3.1.9]',
  'EC:4.1.2.13]',
  'EC:1.1.1.47 3.1.1.31]',
  'EC:2.7.1.11]'])

In [4]:
# insert values into the enzymes table
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
cmd = ''' INSERT INTO enzymes (kegg_id, name, function, ec_number) VALUES(?,?,?,?)'''


kegg_ids,names,functions,ec_numbers = get_values(lines)
for i in range(len(names)):
    name = names[i]
    function = functions[i]
    kegg_id = kegg_ids[i]
    ec_number = ec_numbers[i]
    temp_value = (kegg_id, name, function, ec_number)
    c.execute(cmd,temp_value)
    conn.commit()

In [5]:
#  Build the helper function to search and fetch for gene, nucleotide, and protein
from Bio import Entrez
from Bio import SeqIO
#Entrez.esearch(db,term, sort)
#Entrez.efetch(db, id, rettype, retmode)
#for enzymes and genes only; Entrez doesn't have pathway info

def search_fetch(db_type,term):
    Entrez.email = 'ych323@berkeley.edu'
    handle = Entrez.esearch(db = db_type,
                       term = term,
                       sort= 'relevance',
                       idtype= 'acc')
    fetched_dict = Entrez.read(handle)
    print(fetched_dict)
    if not fetched_dict["IdList"]:
        data = "None"
        return data
    handle = Entrez.efetch(db = db_type, id = fetched_dict["IdList"][0], rettype = 'fasta', retmode = 'text')
    data = handle.read()
    return data

# 'homo sapiens[ORGN] G6PD cds'
# build a term list of all the search "terms species[ORGN] enzyme_name"
# build the 3 * 12 entries of name, organism, and description of the gene
species_list = ["Drosophila","Escherichia coli","Homo sapiens"]
ter_list = [str(species) + "[ORGN]" + " " for species in species_list ]
term_list = [str(term) + str(name) for term in ter_list for name in names]
name_list = [str(name) for term in ter_list for name in names]
organism_list = [str(species) for species in species_list for name in names]
descriptions_list = [str(function) for species in species_list for function in functions]
# print(term_list)
# print(organism_list)
# print(descriptions_list)


In [6]:


def get_nucleotide_sequence(term_list):
    nucleotide_list = []
    for term in term_list:
        print(term)
        data = search_fetch("nucleotide",term)
        if data == "None":
            nucleotide_list.append(data)
        f = open('temp.fa','w')
        f.write(data)
        f.close()
        print("parsed")
        for seq in SeqIO.parse("temp.fa", "fasta"):
            nucleotide_list.append(str(seq.seq))
    return nucleotide_list

nucleotide_list = get_nucleotide_sequence(term_list)
print(nucleotide_list[1:4])

# f = open('temp.fa','w')
# f.write(data)
# f.close()
# print("parsed")
# for seq in SeqIO.parse("temp.fa", "fasta"):
#     print(seq.id)
#     print(seq.seq)

# TODO: Make a function to iterate through the query
# for i in Entrez.read(handle)["IdList"]:
#     handle = Entrez.efetch(db = 'nucleotide', id = i, rettype = 'fasta', retmode = 'text')
#     print(handle.read())

Drosophila[ORGN] IDH3
DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Drosophila[ORGN]', 'To': '"Drosophila"[Organism] OR "Drosophila"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Drosophila"[Organism]', 'Field': 'Organism', 'Count': '1476189', 'Explode': 'Y'}, attributes={}), DictElement({'Term': '"Drosophila"[Organism]', 'Field': 'Organism', 'Count': '204887', 'Explode': 'Y'}, attributes={}), 'OR', 'GROUP', DictElement({'Term': 'IDH3[All Fields]', 'Field': 'All Fields', 'Count': '39', 'Explode': 'N'}, attributes={}), 'AND'], 'QueryTranslation': '("Drosophila"[Organism] OR "Drosophila"[Organism]) AND IDH3[All Fields]'}, attributes={})
parsed
Drosophila[ORGN] G6PD,zwf
DictElement({'Count': '68', 'RetMax': '20', 'RetStart': '0', 'IdList': ['XM_002011542.2', 'CH933814.1', 'U45985.1', 'U44721.1', 'U43167.1', 'U43166.1', 'U43165.1', 'U42749.1', 'U42748.1', 'U42747.1', 'U42746.1', 'U42745.1', 'U42

DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Drosophila[ORGN]', 'To': '"Drosophila"[Organism] OR "Drosophila"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Drosophila"[Organism]', 'Field': 'Organism', 'Count': '1476189', 'Explode': 'Y'}, attributes={}), DictElement({'Term': '"Drosophila"[Organism]', 'Field': 'Organism', 'Count': '204887', 'Explode': 'Y'}, attributes={}), 'OR', 'GROUP', DictElement({'Term': 'fbaB[All Fields]', 'Field': 'All Fields', 'Count': '10728', 'Explode': 'N'}, attributes={}), 'AND'], 'QueryTranslation': '("Drosophila"[Organism] OR "Drosophila"[Organism]) AND fbaB[All Fields]'}, attributes={})
parsed
Drosophila[ORGN] H6PD
DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Drosophila[ORGN]', 'To': '"Drosophila"[Organism] OR "Drosophila"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': 

parsed
Escherichia coli[ORGN] LSC2
DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Escherichia coli[ORGN]', 'To': '"Escherichia coli"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Escherichia coli"[Organism]', 'Field': 'Organism', 'Count': '5153742', 'Explode': 'Y'}, attributes={}), DictElement({'Term': 'LSC2[All Fields]', 'Field': 'All Fields', 'Count': '261', 'Explode': 'N'}, attributes={}), 'AND'], 'QueryTranslation': '"Escherichia coli"[Organism] AND LSC2[All Fields]'}, attributes={})
parsed
Escherichia coli[ORGN] pgi1
DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Escherichia coli[ORGN]', 'To': '"Escherichia coli"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Escherichia coli"[Organism]', 'Field': 'Organism', 'Count': '5153742', 'Explode': 'Y'}, attributes={}), DictElement({'Term': 'pgi1[All Fie

parsed
Homo sapiens[ORGN] rpiA
DictElement({'Count': '30', 'RetMax': '20', 'RetStart': '0', 'IdList': ['KR709393.1', 'KJ902259.1', 'NM_144563.2', 'HF548095.1', 'NG_016710.1', 'AC062029.7', 'AC096579.1', 'FUIG01000070.1', 'NC_000002.12', 'CM000253.1', 'CH471215.1', 'BC015529.1', 'NM_001433.4', 'NT_022184.16', 'GL583303.1', 'FW828349.1', 'FW828348.1', 'FW828347.1', 'FW828346.1', 'FW828345.1'], 'TranslationSet': [DictElement({'From': 'Homo sapiens[ORGN]', 'To': '"Homo sapiens"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Homo sapiens"[Organism]', 'Field': 'Organism', 'Count': '16629335', 'Explode': 'Y'}, attributes={}), DictElement({'Term': 'rpiA[All Fields]', 'Field': 'All Fields', 'Count': '88640', 'Explode': 'N'}, attributes={}), 'AND'], 'QueryTranslation': '"Homo sapiens"[Organism] AND rpiA[All Fields]'}, attributes={})
parsed
Homo sapiens[ORGN] LSC2
DictElement({'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['KC154031.1'], 'TranslationSet': [Dic

In [7]:
from Bio import Entrez
from Bio import SeqIO
#Entrez.esearch(db,term, sort)
#Entrez.efetch(db, id, rettype, retmode)
#for enzymes and genes only; Entrez doesn't have pathway info


def get_protein(term_list):
    protein_list = []
    for term in term_list:
        print(term)
        data = search_fetch("protein",term)
        f = open('temp.fa','w')
        f.write(data)
        f.close()
        print("parsed")
        for seq in SeqIO.parse("temp.fa", "fasta"):
            protein_list.append(seq.seq)
    return protein_list
protein_list = get_protein(term_list)
print(protein_list[0])


Drosophila[ORGN] IDH3
DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Drosophila[ORGN]', 'To': '"Drosophila"[Organism] OR "Drosophila"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Drosophila"[Organism]', 'Field': 'Organism', 'Count': '1002404', 'Explode': 'Y'}, attributes={}), DictElement({'Term': '"Drosophila"[Organism]', 'Field': 'Organism', 'Count': '157507', 'Explode': 'Y'}, attributes={}), 'OR', 'GROUP', DictElement({'Term': 'IDH3[All Fields]', 'Field': 'All Fields', 'Count': '52', 'Explode': 'N'}, attributes={}), 'AND'], 'QueryTranslation': '("Drosophila"[Organism] OR "Drosophila"[Organism]) AND IDH3[All Fields]'}, attributes={})
parsed
Drosophila[ORGN] G6PD,zwf
DictElement({'Count': '77', 'RetMax': '20', 'RetStart': '0', 'IdList': ['XP_002011578.1', 'EDW05568.1', 'P12646.2', 'Q27638.1', 'AAA99107.1', 'AAA99092.1', 'AAA99073.1', 'AAA99072.1', 'AAA99071.1', 'AAB02812.1', 'AAB02811.1', 'A

DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Drosophila[ORGN]', 'To': '"Drosophila"[Organism] OR "Drosophila"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Drosophila"[Organism]', 'Field': 'Organism', 'Count': '1002404', 'Explode': 'Y'}, attributes={}), DictElement({'Term': '"Drosophila"[Organism]', 'Field': 'Organism', 'Count': '157507', 'Explode': 'Y'}, attributes={}), 'OR', 'GROUP', DictElement({'Term': 'fbaB[All Fields]', 'Field': 'All Fields', 'Count': '11291', 'Explode': 'N'}, attributes={}), 'AND'], 'QueryTranslation': '("Drosophila"[Organism] OR "Drosophila"[Organism]) AND fbaB[All Fields]'}, attributes={})
parsed
Drosophila[ORGN] H6PD
DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Drosophila[ORGN]', 'To': '"Drosophila"[Organism] OR "Drosophila"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': 

parsed
Escherichia coli[ORGN] LSC2
DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Escherichia coli[ORGN]', 'To': '"Escherichia coli"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Escherichia coli"[Organism]', 'Field': 'Organism', 'Count': '36879557', 'Explode': 'Y'}, attributes={}), DictElement({'Term': 'LSC2[All Fields]', 'Field': 'All Fields', 'Count': '208', 'Explode': 'N'}, attributes={}), 'AND'], 'QueryTranslation': '"Escherichia coli"[Organism] AND LSC2[All Fields]'}, attributes={})
parsed
Escherichia coli[ORGN] pgi1
DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Escherichia coli[ORGN]', 'To': '"Escherichia coli"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Escherichia coli"[Organism]', 'Field': 'Organism', 'Count': '36879557', 'Explode': 'Y'}, attributes={}), DictElement({'Term': 'pgi1[All F

parsed
Homo sapiens[ORGN] rpiA
DictElement({'Count': '10', 'RetMax': '10', 'RetStart': '0', 'IdList': ['CCO13806.1', 'P49247.3', 'NP_653164.2', 'AAY14778.1', 'AAY24200.1', 'SJM35040.1', 'EAW77067.1', 'EAW77066.1', 'AAH15529.2', 'NP_001424.3'], 'TranslationSet': [DictElement({'From': 'Homo sapiens[ORGN]', 'To': '"Homo sapiens"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Homo sapiens"[Organism]', 'Field': 'Organism', 'Count': '1266755', 'Explode': 'Y'}, attributes={}), DictElement({'Term': 'rpiA[All Fields]', 'Field': 'All Fields', 'Count': '90702', 'Explode': 'N'}, attributes={}), 'AND'], 'QueryTranslation': '"Homo sapiens"[Organism] AND rpiA[All Fields]'}, attributes={})
parsed
Homo sapiens[ORGN] LSC2
DictElement({'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [DictElement({'From': 'Homo sapiens[ORGN]', 'To': '"Homo sapiens"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Homo sapiens"[Organism]', 'F

In [8]:


from Bio import Entrez
#Entrez.esearch(db,term, sort)
#Entrez.efetch(db, id, rettype, retmode)
#for enzymes and genes only; Entrez doesn't have pathway info




Entrez.email = 'ych323@berkeley.edu'
handle = Entrez.esearch(db = 'gene',
                       term = 'homo sapiens[ORGN] G6PD',
                       sort= 'relevance',
                       idtype= 'acc')

fetched_dict = Entrez.read(handle)
print(fetched_dict)

handle = Entrez.efetch(db = 'gene', id = fetched_dict["IdList"][0], rettype = 'fasta', retmode = 'text')
print(handle.read())

DictElement({'Count': '112', 'RetMax': '20', 'RetStart': '0', 'IdList': ['2539', '8266', '9563', '7157', '3569', '4524', '3586', '5743', '6774', '4790', '3458', '54658', '5444', '2147', '2950', '2153', '472', '4780', '3162', '3043'], 'TranslationSet': [DictElement({'From': 'homo sapiens[ORGN]', 'To': '"Homo sapiens"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Homo sapiens"[Organism]', 'Field': 'Organism', 'Count': '224085', 'Explode': 'Y'}, attributes={}), DictElement({'Term': 'G6PD[All Fields]', 'Field': 'All Fields', 'Count': '2253', 'Explode': 'N'}, attributes={}), 'AND'], 'QueryTranslation': '"Homo sapiens"[Organism] AND G6PD[All Fields]'}, attributes={})

1. G6PD
Official Symbol: G6PD and Name: glucose-6-phosphate dehydrogenase [Homo sapiens (human)]
Other Aliases: G6PD1
Other Designations: glucose-6-phosphate 1-dehydrogenase
Chromosome: X; Location: Xq28
Annotation: Chromosome X NC_000023.11 (154531390..154547586, complement)
MIM: 305900
ID: 2539


